<a href="https://colab.research.google.com/github/ayulockin/SwAV-TF/blob/master/SwAV_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

In [3]:
# Architecture utils
def get_resnet_backbone():
    base_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=(None, None, 3))
    base_model.trainabe = True
    
    inputs = Input((None, None, 3))
    h = base_model(inputs, training=True)
    h = GlobalAveragePooling2D()(h)

    backbone = Model(inputs, h)

    return backbone

In [4]:
tf.keras.backend.clear_session()
get_embedding = get_resnet_backbone()

get_embedding.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [5]:
def get_projection_prototype():
  inputs = Input((2048, ))
  projection_1 = Dense(2048)(inputs)
  projection_1 = BatchNormalization()(projection_1)
  projection_1 = Activation("relu")(projection_1)

  projection_2 = Dense(128)(projection_1)

  projection_2_normalize = tf.math.l2_normalize(projection_2, axis=1)
  
  prototype = Dense(3000, use_bias=False)(projection_2_normalize)

  return Model(inputs=inputs, outputs=[projection_2_normalize, prototype])

In [6]:
tf.keras.backend.clear_session()
projection_prototype = get_projection_prototype()

projection_prototype.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 2048)         4196352     input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 2048)         8192        dense[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 2048)         0           batch_normalization[0][0]        
_______________________________________________________________________________________